In [ ]:
# 检测 导出的app.py 文件
file_path = "../app.py"
!flake8 {file_path} --max-line-length=240
!pylint {file_path}

In [ ]:
%%writefile ..\app.py
"""
项目初始化模板

这个文件用于设置项目的基本配置，包括:
1. 设置项目路径
2. 加载环境变量
3. 读取配置文件

注意: 此文件可以通过notebook导出到app.py，也可以直接在notebook中运行
"""
import os
import sys
import json
from dotenv import load_dotenv

# ===== 初始化项目路径 =====
# 根据运行环境(脚本或notebook)获取项目根目录
if "__file__" in globals():
    # 脚本环境: 使用文件的绝对路径
    current_dir = os.path.dirname(os.path.abspath(__file__))
    root_dir = os.path.normpath(os.path.join(current_dir, ".."))
else:
    # Jupyter Notebook环境: 使用当前工作目录
    current_dir = os.getcwd()
    current_dir = os.path.join(current_dir, "..")
    root_dir = os.path.normpath(os.path.join(current_dir))

# 规范化路径并添加到系统路径
current_dir = os.path.normpath(current_dir)
sys.path.append(current_dir)

# 加载.env文件中的环境变量
load_dotenv(dotenv_path=os.path.join(current_dir, ".env"))

# 加载项目配置文件
with open(os.path.join(current_dir, "config.json"), encoding="utf-8") as file:
    config = json.load(file)


In [ ]:
import base64
with open('../adskip.png', 'rb') as f:
    base64_string = base64.b64encode(f.read()).decode('utf-8')
    print(base64_string)

In [ ]:
import qrcode
import qrcode.image.svg
from xml.etree import ElementTree as ET
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

# ====== 位置独立配置source参数 ======
DEFAULT_SOURCE = "homepage"

def add_source_to_url(url, default_source=None):
    """检查url中是否有source参数，如果没有则添加默认source"""
    if default_source is None:
        default_source = DEFAULT_SOURCE
    parsed = urlparse(url)
    query = parse_qs(parsed.query)
    if "source" not in query:
        query["source"] = [default_source]
        new_query = urlencode(query, doseq=True)
        new_url = urlunparse((
            parsed.scheme,
            parsed.netloc,
            parsed.path,
            parsed.params,
            new_query,
            parsed.fragment
        ))
        return new_url
    else:
        return url

def generate_fixed_svg_qrcode(data, color="#000000", bg_color="#FFFFFF", source=None):
    """
    生成固定尺寸的二维码SVG，返回可以直接用于SVG的path和viewBox等信息
    自动尝试合适的二维码版本，保证数据能装下且尺寸最小
    """
    # 检查并补充source参数
    data = add_source_to_url(data, default_source=source)
    print(f"生成二维码: {data}")

    # 自动尝试合适的二维码版本
    min_version = 3
    max_version = 8
    qr = None
    actual_version = None
    for version in range(min_version, max_version + 1):
        try:
            qr = qrcode.QRCode(
                version=version,
                error_correction=qrcode.constants.ERROR_CORRECT_M,
                box_size=1,
                border=1,
            )
            qr.add_data(data)
            qr.make(fit=False)
            actual_version = version
            print(f"使用版本: {version}")
            break
        except Exception as e:
            print(f"版本 {version} 失败: {e}")
            if version == max_version:
                raise e
            continue

    # 生成SVG
    factory = qrcode.image.svg.SvgPathImage
    img = qr.make_image(image_factory=factory,
                        fill_color=color,
                        back_color=bg_color)

    # 获取SVG字符串
    svg_str = img.to_string(encoding='unicode')

    # 解析SVG获取path
    try:
        root = ET.fromstring(svg_str)

        # 计算viewBox尺寸
        version_size = 21 + (actual_version - 1) * 4  # 二维码点阵尺寸
        total_size = version_size + 2  # 加上border
        viewbox = f"0 0 {total_size} {total_size}"
        viewbox_modify = f"0 0 {total_size/10} {total_size/10}"

        # 获取path数据
        path_element = root.find('.//{http://www.w3.org/2000/svg}path')
        if path_element is not None:
            path_data = path_element.get('d', '')
        else:
            path_data = ''

        print(f"viewBox: {viewbox}")
        print(f"Path长度: {len(path_data)} 字符")

        return {
            'path_data': path_data,
            'viewBox': viewbox,
            'viewBox_modify': viewbox_modify,
            'version': actual_version,
            'size': total_size,
            'full_svg': svg_str,
            'color': color
        }

    except Exception as e:
        print(f"解析SVG出错: {e}")
        return None

# 测试使用
if __name__ == "__main__":
    result = generate_fixed_svg_qrcode(
        "https://otokonoizumi.github.io#projects",
        color="#c92a2a",
        bg_color="#FFFFFF",
        source="qq"
    )

    if result:
        print("\n=== 生成结果 ===")
        print(f"使用版本: {result['version']}")
        print(f"实际尺寸: {result['size']}x{result['size']}")
        print(f"viewBox: {result['viewBox']}")
        print(f"path数据: {result['path_data'][:100]}...")

        # 输出可以直接用于SVG的代码
        print("\n=== 用于SVG的代码 ===")
        print(f'    <svg x="10" y="10" width="100" height="100" viewBox="{result["viewBox_modify"]}">')
        print(f'      <path d="{result["path_data"]}"')
        print(f'       fill="{result["color"]}"/>')
        print('    </svg>')